In [120]:
import json

import os

import pandas as pd

import numpy as np

import matplotlib.pyplot as plt

import datetime

def get_name_of_job(name):
    
    counter = 0
        
    for pos, char in enumerate(name):

        if char == '_':

            counter += 1
        
        if counter < 3:
            
            position = pos + 1
            
        if counter == 4:
            
            another_position = pos
            
            break
            
    key = name[0:position:]
    
    make = name[position+1:another_position:]
    
    return key, make

path = os.chdir('/Users/andrlluuk/Desktop/flink_metric_data_new')
#здесь нужно будет прописать свой путь до папки с данными

path = os.getcwd()

struct = {}

cnt = 0

cntr = 0

dict_of_wrong_tables = {}

amount_of_write = 0
        
amount_of_zero_write = 0
        
amount_of_zero_write_set = set()
        
amount_of_read = 0
        
amount_of_zero_read = 0
        
amount_of_zero_read_set = set()


def in_dictionary(key, dict):
    
    if key in dict:
        
        return True
    
    return False



for root, dirs, files in os.walk(path):
    
    for filename in files:
        
        full_path = filename
        
        key_for_dict, type_of_data = get_name_of_job(full_path)
        
        if type_of_data == 'write':
            
            amount_of_write += 1
            
        else:
            
            amount_of_read += 1
        
        with open(full_path) as f:
            
            tmp = json.load(f)
        
        df = tmp['datapoints']
        
        average = np.zeros(len(df))

        time = [] 
        
        for i in range(len(df)):
            
            average[i] = df[i]['average']
            
            date = datetime.datetime.fromtimestamp(df[i]['timestamp']/1000.0)
            
            date = date.strftime('%Y-%m-%d %H:%M:%S')
            
            time.append(date)
            
        if np.sum(average)/len(df) == 0:
            
            if type_of_data == 'write':
                
                amount_of_zero_write += 1
                
                amount_of_zero_write_set.add(key_for_dict)
                
            else:
                
                amount_of_zero_read += 1
                
                amount_of_zero_read_set.add(key_for_dict)
                
        if not in_dictionary(key_for_dict, struct):
            
            table = pd.DataFrame({type_of_data : average, 'timestamp': time})
            
            struct[key_for_dict] = table
            
            cnt += 1
            
            dict_of_wrong_tables[key_for_dict] = 2
            
            
        else:
            
            table = pd.DataFrame({type_of_data : average})
            
            struct[key_for_dict] = pd.concat([struct[key_for_dict], table], axis = 1)
            
            columnsTitles = ['read', 'write', 'timestamp']

            struct[key_for_dict] = struct[key_for_dict].reindex(columns=columnsTitles)
            
            cntr += 1
            
            dict_of_wrong_tables[key_for_dict] = 3

In [123]:
print('All job:',cnt)

print('One way job:',cntr)

value = 2

print('Next jobs with only "read" function:' , end = '')

for key, val in dict_of_wrong_tables.items():
    
    if val == value:
        
        print(key)
        
print('Amount of readers:', amount_of_read)
      
print('Amount of empty readers:', amount_of_zero_read)
      
print('All empty readers:', amount_of_zero_read_set)
        
print('___________________________________________________________')

print('Amount of writers:', amount_of_write)

print('Amount of empty writers:', amount_of_zero_write)

print('All empty writers', amount_of_zero_write_set)

print('___________________________________________________________')

print('Only empty readers:',amount_of_zero_read_set - amount_of_zero_write_set)

print('___________________________________________________________')

print('Only zempty writers:',amount_of_zero_write_set - amount_of_zero_read_set)

print('___________________________________________________________')

print('All empty zeros or writers:', amount_of_zero_write_set ^ amount_of_zero_read_set)

All job: 396
One way job: 394
Next jobs with only "read" function:i8e28d444g2144i0d94384efd5802gfd_14612_203106
i8e28d444g2144i0d94384efd5802gfd_17009_206351
Amount of readers: 396
Amount of empty readers: 152
All empty readers: {'8888e5di42ef4ei4858iff348gdee51g_19791_185293', '1dh1e3d93h944idh991i576e48917264_13604_130577', 'gi1247h29fe3449184hhfe8i9960457d_14867_207836', '8888e5di42ef4ei4858iff348gdee51g_19761_184029', '8888e5di42ef4ei4858iff348gdee51g_19761_184025', 'i6dg8515940i42f488h927if637d56i4_13438_132252', '8888e5di42ef4ei4858iff348gdee51g_19791_185291', 'd7h8e83ief014e59dii5id3ie14585i2_13599_184342', '8080000000017000000_14788_167456', '1dh1e3d93h944idh991i576e48917264_13604_129574', 'i6dg8515940i42f488h927if637d56i4_13438_130261', 'i8e28d444g2144i0d94384efd5802gfd_14612_191585', 'gi1247h29fe3449184hhfe8i9960457d_15740_205510', '8888e5di42ef4ei4858iff348gdee51g_19761_184028', '10h4id8fd1ii4477diifdd5d3iig9596_15360_152317', '6d6010eh7716405490hi5047ffehe0g4_14393_153395',